In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from nb_004 import *

In [ ]:
DATA_PATH = Path('../../data')
PATH = DATA_PATH/'caltech101'

In [ ]:
data_mean,data_std = map(tensor, ([0.5355,0.5430,0.5280], [0.2909,0.2788,0.2979]))

In [ ]:
#export
class FilesDataset(Dataset):
    def __init__(self, fns, labels, classes=None):
        if classes is None: classes = list(set(labels))
        self.classes = classes
        self.class2idx = {v:k for k,v in enumerate(classes)}
        self.fns = np.array(fns)
        self.y = [self.class2idx[o] for o in labels]
        
    @classmethod
    def from_folder(cls, folder, classes=None, test_pct=0., tfms=None):
        if classes is None: classes = [cls.name for cls in find_classes(folder)]
            
        fns,labels = [],[]
        for cl in classes:
            fnames = get_image_files(folder/cl)
            fns += fnames
            labels += [cl] * len(fnames)
            
        if test_pct==0.: return cls(fns, labels)
        fns,labels = np.array(fns),np.array(labels)
        is_test = np.random.uniform(size=(len(fns),)) < test_pct
        return cls(fns[~is_test], labels[~is_test]), cls(fns[is_test], labels[is_test])

    def __len__(self): return len(self.fns)

    def __getitem__(self,i):
        x = PIL.Image.open(self.fns[i]).convert('RGB')
        x = pil2tensor(x)
        return x,self.y[i]

In [ ]:
#classes = ['airplanes','Motorbikes','Faces','watch','Leopards']
np.random.seed(42)
train_ds,valid_ds = FilesDataset.from_folder(PATH, test_pct=0.2)
classes = train_ds.classes

In [ ]:
@reg_affine
def zoom1(scale: uniform = 1.0, row_pct:uniform = 0.5, col_pct:uniform = 0.5) -> TfmType.Affine:
    s = 1-math.sqrt(scale)
    col_c = s * (2*col_pct - 1)
    row_c = s * (2*row_pct - 1)
    return get_zoom_mat(math.sqrt(scale), math.sqrt(scale), col_c, row_c)

In [ ]:
@reg_affine
def squish(scale: uniform = 1.0, row_pct:uniform = 0.5, col_pct:uniform = 0.5) -> TfmType.Affine:
    if scale <= 1: 
        col_c = (1-scale) * (2*col_pct - 1)
        return get_zoom_mat(scale, 1, col_c, 0.)
    else:          
        row_c = (1-1/scale) * (2*row_pct - 1)
        return get_zoom_mat(1, 1/scale, 0., row_c)

In [ ]:
sz = 224
trn_tfms = [squish_tfm(scale=(0.75,1.33), row_pct=(0,1.), col_pct=(0,1.)),
            zoom1_tfm(scale=(0.8,1.), row_pct=(0,1.), col_pct=(0,1.)),
            flip_lr_tfm(p=0.5),
            crop_tfm(size=sz),
            normalize_tfm(mean=data_mean,std=data_std)] #torchvision.transforms.RandomRotation(10),
val_tfms = [crop_tfm(size=sz),
            normalize_tfm(mean=data_mean,std=data_std)]

In [ ]:
#classes = ['airplanes','Motorbikes','Faces','watch','Leopards']
np.random.seed(42)
train_ds,valid_ds = FilesDataset.from_folder(PATH, test_pct=0.2)
classes = train_ds.classes

In [ ]:
train_ds = TfmDataset(train_ds, trn_tfms, size=224, do_crop=True)
valid_ds = TfmDataset(valid_ds, val_tfms, size=224, do_crop=True)

In [ ]:
x,y = train_ds[0]
x,y = valid_ds[0]

In [ ]:
plt.imshow(x.numpy().transpose(1,2,0))

In [ ]:
x.size()

In [ ]:
class DataBunch():
    "Data object that regroups training and validation data"
    
    def __init__(self, train_ds:Dataset, valid_ds:Dataset, bs:int=64, device:torch.device=None, num_workers:int=4):
        self.device,self.bs = default_device if device is None else device,bs
        self.train_dl = DeviceDataLoader.create(train_ds, bs, shuffle=True, num_workers=num_workers, device=self.device)
        self.valid_dl = DeviceDataLoader.create(valid_ds, bs*2, shuffle=False, num_workers=num_workers, device=self.device)

    def __repr__(self) -> str:
        res = f'DataBunch, batch_size={self.bs} on {self.device}.\n  train dataloader: {len(self.train_dl)} batches'
        if self.valid_dl is not None: res += f'\n  validation dataloader: {len(self.valid_dl)} batches'
        return res

    #TODO: uncomment when transforms are available
    #@classmethod
    #def create(cls, train_ds, valid_ds, train_tfm=None, valid_tfm=None, **kwargs):
    #    return cls(TfmDataset(train_ds, train_tfm), TfmDataset(valid_ds, valid_tfm))
        
    @property
    def train_ds(self) -> Dataset: return self.train_dl.dl.dataset
    @property
    def valid_ds(self) -> Dataset: return self.valid_dl.dl.dataset

In [ ]:
data = DataBunch(train_ds, valid_ds, bs=64, num_workers=8)

In [ ]:
x,y = next(iter(data.valid_dl))

_,axes = plt.subplots(2,4, figsize=(9,3))
for i,ax in enumerate(axes.flat): show_image(x[i], ax)

## Train

In [ ]:
@dataclass
class Learner():
    "Object that wraps together some data, a model, a loss function and an optimizer"
    
    data:DataBunch
    model:nn.Module
    opt_fn:Callable=optim.SGD
    loss_fn:Callable=F.cross_entropy
    metrics:Collection[Callable]=None
    true_wd:bool=False
    def __post_init__(self): 
        self.model = self.model.to(self.data.device)
        self.callbacks = []

    def fit(self, epochs:int, lr:float, wd:float=0., callbacks:Collection[Callback]=None):
        if not hasattr(self, 'opt'): self.create_opt(lr, wd)
        if callbacks is None: callbacks = []
        callbacks = self.callbacks + callbacks
        fit(epochs, self.model, self.loss_fn, self.opt, self.data, callbacks=callbacks, metrics=self.metrics)
    
    def create_opt(self, lr:float, wd:float=0.):
        self.opt = OptimWrapper(self.opt_fn(self.model.parameters(), lr), wd=wd, true_wd=self.true_wd)
        self.recorder = Recorder(self.opt, self.data.train_dl)
        self.callbacks = [self.recorder] + self.callbacks

In [ ]:
model = Darknet([1, 2, 4, 6, 3], num_classes=len(classes), nf=16).cuda()
learn = Learner(data, model)
learn.loss_fn = F.cross_entropy
learn.metrics = [accuracy]
learn.opt_fn = partial(optim.Adam, betas=(0.95,0.99))
learn.true_wd = True

In [ ]:
def fit_one_cycle(learn:Learner, max_lr:float, cyc_len:int, moms=(0.95,0.85), div_factor:float=10.,
                 pct_end:float=0.1, wd:float=0.):
    "Fits a model following the 1cycle policy"
    cbs = [OneCycleScheduler(learn, max_lr, cyc_len, moms, div_factor, pct_end)]
    learn.fit(cyc_len, max_lr/div_factor, wd=wd, callbacks=cbs)

In [ ]:
model.layers[0][1].weight

In [ ]:
fit_one_cycle(learn, 4e-3, 5, wd=0.1)

In [ ]:
model.layers[0][1].weight

In [ ]:
torch.save(learn.model.state_dict(),PATH/'model.pt')

In [ ]:
torch.save(model.state_dict(),PATH/'model1.pt')

In [ ]:
model.eval()
with torch.no_grad():
    for xb,yb in learn.data.valid_dl:
        print(loss_batch(model, xb, yb, learn.loss_fn, metrics=learn.metrics))

## Eval

In [ ]:
model1 = Darknet([1, 2, 4, 6, 3], num_classes=len(classes), nf=16).cuda()
learn1 = Learner(data, model1)
learn1.loss_fn = F.cross_entropy
learn1.metrics = [accuracy]
learn1.opt_fn = partial(optim.Adam, betas=(0.95,0.99))
learn1.true_wd = True

In [ ]:
model1.layers[0][1].weight

In [ ]:
model1.load_state_dict(torch.load(PATH/'model1.pt'))

In [ ]:
model1.layers[0][1].weight

In [ ]:
model1.eval()
with torch.no_grad():
    *val_metrics,nums = zip(*[loss_batch(learn1.model, xb, yb, learn1.loss_fn, metrics=learn1.metrics)
                                for xb,yb in learn1.data.valid_dl])
    val_metrics = [np.sum(np.multiply(val,nums)) / np.sum(nums) for val in val_metrics]

In [ ]:
val_metrics

In [ ]:
model1.eval()
with torch.no_grad():
    for xb,yb in learn1.data.valid_dl:
        print(loss_batch(model1, xb, yb, learn1.loss_fn, metrics=learn1.metrics))